<img src="../images/logos/wradlib_logo.svg.png" width=250 alt="wradlib logo png" align="left"></img>

# wradlib time series data and quasi vertical profiles

---

## Overview

Within this notebook, we will cover:

1. Reading radar sweep timeseries data into xarray based RadarVolume
1. Examination of RadarVolume and Sweep
1. Calculation of Quasivertical Profiles and Plotting

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Matplotlib Basics](https://foundations.projectpythia.org/core/matplotlib/matplotlib-basics.html) | Helpful | Basic Plotting |
| [Xarray Basics](https://tutorial.xarray.dev/intro.html) | Helpful | Basic Dataset/DataArray |
| [Xarray Plotting](https://tutorial.xarray.dev/intro.html) | Helpful | Basic Plotting/Faceting |

- **Time to learn**: 7.5 minutes

---

## Imports

In [ ]:
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from tqdm import tqdm_notebook as tqdm

import wradlib as wrl

## Import Australian Radar Data

It is assumed, that data from IDR71 (Terrey Hills, Sidney) from 20th of December 2018 is used in this notebook.

In [ ]:
fglob = "data/hdf5/terrey_*.h5"
idr71 = glob.glob(fglob)
idr71.sort()
print("Files available: {}".format(len(idr71)))

## Single Quasi Vertical Profile (QVP)


In [ ]:
odh = wrl.io.open_odim_dataset(idr71[24])

In [ ]:
display(odh)

This example shows how to create a so called QVP. We need to define a function to add a height coordinate for plotting.

In [ ]:
def add_height(ds):
    ds = ds.pipe(wrl.georef.georeference_dataset)
    height = ds.z.mean("azimuth")
    ds = ds.assign_coords({"height": (["range"], height.data)})
    return ds

Here we add the height coordinate and calculate the `mean` over the azimuth using the sweep with the highest available elevation.

In [ ]:
swp = odh[0].pipe(add_height)
display(swp)

In [ ]:
qvp = swp.mean("azimuth")
qvp

In [ ]:
qvp.DBZH.plot(y="height", figsize=(5, 10))

## TimeSeries QVP

All wradlib xarray backends have the capability to read multiple sweeps/volumes in one go. We have to prepare the list of files a bit, though.

In [ ]:
ts = xr.open_mfdataset(
    idr71,
    engine="odim",
    group="dataset1",
    combine="nested",
    concat_dim="time",
)

In [ ]:
display(ts)

### Georeference and add height coordinate

In [ ]:
ts = ts.pipe(add_height)
display(ts)

### Calculate Statistics

In [ ]:
stats = ["median", "mean", "min", "max"]
stat = [
    getattr(ts.where(ts.RHOHV > 0.8), st)("azimuth", skipna=True, keep_attrs=True)
    for st in stats
]
ts_stats = xr.concat(stat, dim="stats")
ts_stats = ts_stats.assign_coords({"stats": stats})

In [ ]:
display(ts_stats)

### Plot QVP's

In [ ]:
levels = np.arange(-30, 80, 5)
facet = ts_stats.TH.plot(
    x="time",
    y="height",
    col="stats",
    col_wrap=2,
    cmap="turbo",
    figsize=(12, 10),
    levels=levels,
)

---

## Summary
Easy creation of Quasi Vertical Profiles was shown.


## Resources and references

- [xarray](https://docs.xarray.dev)
- [dask](https://docs.dask.org/)
- [wradlib xarray backends](https://docs.wradlib.org/en/stable/notebooks/fileio/wradlib_xarray_backends.html)
- [OPERA ODIM_H5](https://www.eumetnet.eu/activities/observations-programme/current-activities/opera/)
- [WMO JET-OWR](https://community.wmo.int/governance/commission-membership/commission-observation-infrastructure-and-information-systems-infcom/commission-infrastructure-officers/infcom-management-group/standing-committee-measurements-instrumentation-and-traceability-sc-mint/joint-expert-team)